# 1. Creating Database

In [13]:
# 📌 Step 1: Install Dependencies
!pip install psycopg2-binary pandas
!pip install pg8000 cloud-sql-python-connector



In [4]:
# 📌 Step 2: Import Libraries
import psycopg2
import pandas as pd
import os


In [1]:
# 📌 Step 2: Import Libraries
import psycopg2
import pandas as pd
import os

# 📌 Step 3: Set Up Database Connection (Replace with Your Details)
DATABASE_URL = "dbname='postgres' user='postgres' host='34.59.119.208' password='avantichhaya'"

def connect_db():
    try:
        conn = psycopg2.connect(DATABASE_URL)
        print("✅ Connected to PostgreSQL Cloud SQL")
        return conn
    except Exception as e:
        print("🚨 Connection Error:", e)
        return None

conn = connect_db()
cursor = conn.cursor()


✅ Connected to PostgreSQL Cloud SQL


In [19]:
# 📌 Step 4: Create Tables (If Not Exists)
create_tables_query = """
CREATE TABLE IF NOT EXISTS emails (
    email_id SERIAL PRIMARY KEY,
    sender_email VARCHAR(255) NOT NULL,
    subject TEXT NOT NULL,
    body TEXT NOT NULL,
    received_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    classified_category VARCHAR(100),
    status VARCHAR(50) DEFAULT 'pending'
);

CREATE TABLE IF NOT EXISTS classification_rules (
    rule_id SERIAL PRIMARY KEY,
    category VARCHAR(100) NOT NULL,
    keywords TEXT[] NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS responses (
    response_id SERIAL PRIMARY KEY,
    email_id INT REFERENCES emails(email_id) ON DELETE CASCADE,
    response_text TEXT NOT NULL,
    responded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS escalations (
    escalation_id SERIAL PRIMARY KEY,
    email_id INT REFERENCES emails(email_id) ON DELETE CASCADE,
    reason TEXT NOT NULL,
    escalated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    resolved BOOLEAN DEFAULT FALSE
);
"""

cursor.execute(create_tables_query)
conn.commit()
print("✅ Tables Created Successfully")


✅ Tables Created Successfully


In [20]:
# 📌 Step 5: Insert Sample Data
insert_data_query = """
INSERT INTO emails (sender_email, subject, body, classified_category, status)
VALUES 
('user1@example.com', 'Leave Request', 'I need leave for 2 days.', 'HR', 'pending'),
('user2@example.com', 'Technical Issue', 'My laptop is not working.', 'IT', 'pending'),
('user3@example.com', 'Salary Inquiry', 'When will I receive my salary?', 'Finance', 'pending')
RETURNING email_id;
"""

cursor.execute(insert_data_query)
conn.commit()
print("✅ Sample Data Inserted")


✅ Sample Data Inserted


In [41]:
# 📌 Step 5: Insert Sample Data
insert_data_query = """
INSERT INTO emails (sender_email, subject, body, classified_category, status)
VALUES 
('user4@example.com', 'Leave Request', 'I need leave for 2 days.', 'aa', 'pending'),
('user5@example.com', 'Technical Issue', 'My laptop is not working.', 'aa', 'pending'),
('user6@example.com', 'Salary Inquiry', 'When will I receive my salary?', 'aa', 'pending')
RETURNING email_id;
"""

cursor.execute(insert_data_query)
conn.commit()
print("✅ Sample Data Inserted")

✅ Sample Data Inserted


In [12]:
# 📌 Step 5: Insert Sample Data
insert_data_query = """
INSERT INTO emails (sender_email, subject, body, classified_category, status)
VALUES 
('avanti.com', 'help needed', 'how does this work? Help me', '', 'pending')
RETURNING email_id;
"""

cursor.execute(insert_data_query)
conn.commit()
print("✅ Sample Data Inserted")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [6]:
alter_query = """
ALTER TABLE emails
ADD COLUMN thread_id INT;
"""

cursor.execute(alter_query)
conn.commit()
print("✅ thread_id column added.")


✅ thread_id column added.


In [3]:
alter_query ="""
    ALTER TABLE emails
    ADD COLUMN classification_confidence FLOAT;
"""
cursor.execute(alter_query)
conn.commit()

print("✅ Added classification_confidence column to emails table.")


✅ Added classification_confidence column to emails table.


In [2]:
alter_query ="""
    ALTER TABLE emails
    ADD COLUMN response_email TEXT;
"""
cursor.execute(alter_query)
conn.commit()

print("✅ Added response_email column to emails table.")


✅ Added response_email column to emails table.


In [7]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_45688\2955623006.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence
0,5,user5@example.com,Technical Issue,My laptop is not working.,2025-03-26 17:08:23.385211,IT & Access Issues,pending,False,NaN,NaN
1,1,user1@example.com,Leave Request,I need leave for 2 days.,2025-03-25 18:53:05.449614,Leave Request,pending,False,NaN,NaN
2,2,user2@example.com,Technical Issue,My laptop is not working.,2025-03-25 18:53:05.449614,IT & Access Issues,pending,False,NaN,NaN
3,3,user3@example.com,Salary Inquiry,When will I receive my salary?,2025-03-25 18:53:05.449614,Payroll Inquiry,pending,False,NaN,NaN
4,15,chhayatundwal192@gmail.com,Payroll discrepancy,My last salary payment seems to be incorrect.,2025-05-04 02:06:38.000000,UNCLASSIFIED,NEW,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
94,124,user24@example.com,Dental plan information,Could you provide details about the dental plan?,2025-03-30 10:55:00.000000,Benefits Inquiry,pending,False,211.0,0.263188
95,119,user19@example.com,Overtime payment inquiry,Was my overtime correctly compensated?,2025-03-30 12:39:00.000000,Expense Reimbursement,pending,False,209.0,0.436998
96,112,user12@example.com,Negotiating offer terms,I would like to discuss the offered compensati...,2025-03-31 11:32:00.000000,Job Offer,pending,True,206.0,0.179425
97,144,user44@example.com,Leave accrual policy,How does the leave accrual system work?,2025-03-31 12:33:00.000000,Policy Clarification,pending,False,220.0,0.529055


In [9]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where sender_email= 'avanaya3@gmail.com';", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_45688\843988559.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where sender_email= 'avanaya3@gmail.com';", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence
0,21,avanaya3@gmail.com,Urgent,I need to do this blah blah.,2025-05-04 03:24:37,None,NOT RESPONDED,False,None,None


In [10]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where email_id>30 and email_id<40;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_45688\3069498300.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where email_id>30 and email_id<40;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence
0,31,newsletters@audible.com,"Short, sweet, and right up your alley.",audible - an amazon company https://www.audibl...,2024-11-25 08:16:13,None,NOT RESPONDED,False,None,None
1,32,email@market.temuemail.com,Please accept: 2 Free Item(s),----------------------------------------------...,2024-11-25 01:10:48,None,NOT RESPONDED,False,None,None
2,33,email@market.temuemail.com,Please Accept: $15 OFF*,----------------------------------------------...,2024-11-22 01:15:27,None,NOT RESPONDED,False,None,None
3,34,email@market.temuemail.com,A massive thank you! $6 OFF for you!,----------------------------------------------...,2024-11-20 01:17:09,None,NOT RESPONDED,False,None,None
4,35,email@market.temuemail.com,Sincerely regret the delay.,----------------------------------------------...,2024-11-19 01:18:40,None,NOT RESPONDED,False,None,None
5,36,CloudPlatform-noreply@google.com,[Update] Changes to the Google Cloud Third-Par...,"Hello Chhau, We are writing to let you know ab...",2024-11-18 18:01:49,None,NOT RESPONDED,False,None,None
6,37,email@market.temuemail.com,Hey friend! You've received the [$6 OFF]!,----------------------------------------------...,2024-11-18 01:21:06,None,NOT RESPONDED,False,None,None
